In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# --- CONFIGURACIÓN INICIAL ---
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

def buscar_archivo(palabra_clave):
    """
    Busca automáticamente el archivo correcto en la carpeta, 
    sin importar si se llama 'clientes.csv' o 'clientes.xlsx - clientes.csv.csv'
    """
    archivos = os.listdir()
    for archivo in archivos:
        if palabra_clave in archivo and archivo.endswith(".csv"):
            return archivo
    return None

def cargar_y_procesar_datos():
    """
    Carga los datos CSV y realiza las fusiones para el Sprint 3.
    """    
    print("🔄 Iniciando carga de datos...")
    
    # 1. CARGA (Adaptado para buscar tus archivos CSV)
    try:
        # Buscamos los archivos automáticamente
        file_cli = buscar_archivo('clientes')
        file_det = buscar_archivo('detalle')
        file_prod = buscar_archivo('productos')
        file_ven = buscar_archivo('ventas')

        if not all([file_cli, file_det, file_prod, file_ven]):
            print("❌ Error: Faltan archivos .csv en la carpeta.")
            return None

        # Leemos usando latin-1 para evitar errores de tildes
        df_cli = pd.read_csv(file_cli, encoding='latin-1')
        df_det = pd.read_csv(file_det, encoding='latin-1')
        df_prod = pd.read_csv(file_prod, encoding='latin-1')
        df_ven = pd.read_csv(file_ven, encoding='latin-1')
        
    except Exception as e:
        print(f"❌ Error al leer archivos: {e}")
        return None

    # 2. CONVERSIÓN DE FECHAS
    df_ven['fecha'] = pd.to_datetime(df_ven['fecha'])

    # 3. FUSIÓN DE DATOS (DATA MERGING)
    print("🔄 Unificando tablas...")

    # Paso A: Unir Detalle con Ventas (Base Transaccional)
    df_master = pd.merge(df_det, df_ven, on='id_venta', how='inner')

    # Paso B: Unir con Productos (Traer Categoría)
    # Usamos suffixes para evitar error si se repite 'nombre_producto'
    df_master = pd.merge(df_master, df_prod, on='id_producto', how='left', suffixes=('', '_prod_dup'))

    # Paso C: Unir con Clientes (Traer Ciudad)
    df_master = pd.merge(df_master, df_cli, on='id_cliente', how='left', suffixes=('', '_cli_dup'))

    print(f"✅ Dataset Maestro creado con {df_master.shape[0]} registros.")
    return df_master

def analizar_categorias(df):
    """
    Tarea Sprint 3: Ingresos por Categoría de Producto.
    """
    print("\n📦 Analizando Categorías...")
    # Agrupamos por categoría
    data = df.groupby('categoria')['importe'].sum().sort_values(ascending=False)

    plt.figure()
    ax = sns.barplot(x=data.index, y=data.values, palette='viridis', hue=data.index, legend=False)
    
    plt.title('Ingresos Totales por Categoría', fontweight='bold')
    plt.ylabel('Facturación ($)')
    plt.xlabel('Categoría')
    
    # Etiqueta de valor sobre las barras
    for i, v in enumerate(data.values):
        ax.text(i, v, f"${v:,.0f}", ha='center', va='bottom', fontsize=10)
        
    plt.show()

def analizar_medios_pago(df):
    """
    Tarea Sprint 3: Preferencia de Medios de Pago.
    Nota: Se eliminan duplicados de venta para contar transacciones reales.
    """
    print("\n💳 Analizando Medios de Pago...")
    
    # Filtramos para tener 1 fila por venta (no por producto)
    transacciones = df.drop_duplicates(subset='id_venta')
    data = transacciones['medio_pago'].value_counts()

    plt.figure(figsize=(8, 8))
    colores = sns.color_palette('pastel')
    
    plt.pie(data, labels=data.index, autopct='%1.1f%%', startangle=140, colors=colores, pctdistance=0.85)
    
    # Círculo blanco para diseño "Donut"
    centre_circle = plt.Circle((0,0),0.70,fc='white')
    fig = plt.gcf()
    fig.gca().add_artist(centre_circle)
    
    plt.title('Distribución de Transacciones por Medio de Pago', fontweight='bold')
    plt.show()

def analizar_geografia(df):
    """
    Tarea Sprint 3: Ventas por Ciudad.
    """
    print("\n📍 Analizando Geografía...")
    
    data = df.groupby('ciudad')['importe'].sum().sort_values(ascending=False)

    plt.figure()
    sns.barplot(x=data.values, y=data.index, palette='coolwarm', hue=data.index, legend=False)
    
    plt.title('Ranking de Ventas por Ciudad', fontweight='bold')
    plt.xlabel('Total Vendido ($)')
    plt.show()

# --- EJECUCIÓN ---
if __name__ == "__main__":
    df_maestro = cargar_y_procesar_datos()
    
    if df_maestro is not None:
        # Validamos que existan las columnas nuevas necesarias (categoria, ciudad, medio_pago)
        cols_necesarias = ['categoria', 'ciudad', 'medio_pago', 'importe']
        
        if all(col in df_maestro.columns for col in cols_necesarias):
            analizar_categorias(df_maestro)
            analizar_medios_pago(df_maestro)
            analizar_geografia(df_maestro)
        else:
            print("❌ Error crítico: Faltan columnas clave para el Sprint 3.")
            print("Columnas actuales:", df_maestro.columns.tolist())

🚀 SPRINT 3: Iniciando carga de datos...
📂 Buscando en carpeta: c:\Users\Miguel\Desktop\Miguelon\GH\Formacion_IA_DataScience_ML\Sprint-3


⛔ ERROR CRÍTICO DE EJECUCIÓN: ❌ No se encontró ningún archivo que contenga la palabra 'clientes' en esta carpeta.
💡 TIP: Verifica que tus 4 archivos .csv estén en la misma carpeta que este Notebook.
